# Lecture 8: Linear Regresion and the Bias-Variance Trade-Off
***

<img src="figs/targetsBannerTry.png" width=1100 height=50>

**Reminder**:  Go to the botttom of the notebook and shift-enter the helper functions.
***

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

### Problem 1: Linear Regression on Dam Data 
***

The data we will explore in this notebook relates the change in the water level in a damn to the rate of flow of water through the damn. First we'll load the data which is stored in a serialized format in the data directory.  We'll store the features in a 2D Numpy array $X$ and the response in a 1D Numpy array $y$. 

In [ ]:
import pickle
with open("data/dam_regression.pickle", "rb") as fname: dam_data = pickle.load(fname)
X = dam_data["features"]
y = dam_data["response"]

If we explore the data a bit we'll see that there are 54 examples in the data set with a single feature. 

In [ ]:
n_examples = len(y)
n_features = X.shape[1]
print("There are {:2d} examples in the data set".format(n_examples))
print("Each example contains {:d} feature(s)".format(n_features))

Next we'll look at the data by making a scatter plot.  The following function (found at the bottom of this notebook) will do all of the plotting for you. 

In [ ]:
dam_plot([(X, y, "data")])

**Part A**: How would you characterize relationship between the feature $X$ and the response $y$? Is the relationship linear?  Is it nonlinear?  Is it highly nonlinear? 


Now we'll use Scikit-Learn to build a simple linear regression model for our data. As usual, the first step in our process will be to divide the data into a training and a validation set.  Scikit-learn has a handy function to do this for us called [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html). Clicking the link will take you to the documentation.  You'll see in the documentation that the function takes in the complete data matrix and response vector, as well as a specification of the fraction of data to go in the training set.  Finally, it takes a `random_state` value that sets the seed of the random number generator.  We'll specify a random state so that everyone's data looks the same.   

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=1734)

Note that we've selected a split with $80\%$ and $20\%$ of the data split into the training set and validation set, respectively.  Executing the following cell will plot the training and validation data together. 

In [ ]:
dam_plot([(X_train, y_train, "training"), (X_valid, y_valid, "validation")])

Next we'll build a linear regression model to fit the data using Scikit-Learn's [LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class. Use the link to check out the documentation. 

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()

So far we've only created an instance of the class.  Next we need to fit the model to our data.  Almost all models in Scikit-Learn come with a `.fit` method used to train the model.  We need to pass in the training features in `X_train` and the training responses in `y_train`. 

In [ ]:
linreg.fit(X_train, y_train)

OK, so we've trained our linear regression with Scikit-Learn.  But how successful were we?  One way we might check is by looking at the MSE on the training data and on the validation data. In order to compute the MSE we first need to compute the predictions of the model on the training data and the validation data. We can do this using the `predict` method and passing in the training features and the validation features stored in `X_train` and `X_valid`.  

In [ ]:
yhat_train = linreg.predict(X_train)
yhat_valid = linreg.predict(X_valid)

Finally, we can use the `mean_square_error` method to compute the MSE of the predictions.  

In [ ]:
from sklearn.metrics import mean_squared_error

mse_train = mean_squared_error(yhat_train, y_train)
mse_valid = mean_squared_error(yhat_valid, y_valid)

print("Training MSE:   {:.3f}".format(mse_train))
print("Validation MSE: {:.3f}".format(mse_valid))

**Part B**: The MSE is a relatively simple calculation that potentially doesn't warrant a call to an sklearn function.  Can you verify these values by computing them directly in Numpy? Remember that the formula for MSE is given by 

$$
MSE = \frac{1}{n} \sum_{i=1}^n \left( y_i - \hat{y}_i\right)^2
$$

In [ ]:
my_mse_train = 0.0 #TODO 
my_mse_valid = 0.0 #TODO 
print("Training MSE:   {:.3f}".format(my_mse_train))
print("Validation MSE: {:.3f}".format(my_mse_valid))

**Part C**: Finally, let's plot our linear model against our training and validation data.  Like most plotting libraries, Matplotlib takes in an arrays of $x$-values and $y$-values and simply connects the dots.  In order to generate some plotting data, one thing we could do is create an array of equispaced $x$-values an then use our model prediction to get an array of $y$-values.  Fill in the missing code below to accomplish this. 

**Note**: The quirky `.reshape(-1,1)` call below turns the 1D array that `linespace` returns into a 2D array.  Pretty uniformly across Scikit-Learn classes, the `fit` and `predict` methods expect your features to be a 2D array.  Learn this quirk now to avoid many painful `DeprecationWarning`'s in the future. 

In [ ]:
xplot = np.linspace(-60,60,100).reshape(-1,1)
yplot = np.zeros_like(xplot) # TODO - Fix This  
dam_plot([(X_train, y_train, "training"), (X_valid, y_valid, "validation")], [(xplot, yplot, "model")])

As expected, the linear model does not seem to fit the data particularly well.  Let's see if we can improve this with a polynomial model. 

### Problem 2: Building a Polynomial Regression Model in Scikit-Learn 
***

Recall that Polynomial Regression is just multiple linear regression where we create new features that are powers of our original single feature.  In other words, our single feature $X$ turns into multiple features as 

$$
    X \quad \mapsto \quad X_1 = X, \quad X_2 = X^2, \quad X_3 = X^3 ,\quad \ldots,\quad X_p = X^p 
$$

We could create a new Numpy array where each column is a power of our original feature $X$, but Scikit-Learn can do this much more efficiently using the [PolynomialFeatures](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) class. We'll demonstrate it's functionality first on a very simple example and then apply it to our data.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

Z = np.array([[1], [2], [3]])
print(Z)

Note that `Z` is just a 2D Numpy array with $1$, $2$, and $3$ as the entries in it's sole column.  Next we'll create an instance of PolynomialFeatures that augments `Z` with polynomial features up to degree 3. 

In [ ]:
cubic_features = PolynomialFeatures(degree=3, include_bias=False)
Zp = cubic_features.fit_transform(Z)
print(Zp)

You can see that the resulting `Zp` array includes the square and cubes of the original vector as new columns.  Note that if you change the `include_bias` flag to `True` it will prepend a column of ones, making our new matrix a regression design matrix.  We will skip this though because most regression classes in Scikit-Learn will do this for us internally (this is the `fit_intercept=True` flag in `Ridge` described previously). 

**Part A**: OK, time to fit our polynomial regression model.  We'll do this by transforming our training set into a matrix of polynomial features, and then fitting a model using the `LinearRegression` class. For now, we'll start with a degree $3$ polynomial.  

In [ ]:
degree = 3 
poly_features = PolynomialFeatures(degree=degree, include_bias=False)
Xp_train = poly_features.fit_transform(X_train)
polyreg = LinearRegression()
polyreg.fit(Xp_train, y_train)

**Part B**: Let's use our new model to make predictions on the training set and the validation set and computing the resulting MSEs. Note that our Polynomial Regression model expects a matrix of polynomial features, thus we have to transform our validation set features as well.  After that we'll make a plot of the resulting polynomial model. 

In [ ]:
Xp_valid = poly_features.transform(X_valid)
yphat_train = polyreg.predict(Xp_train)
yphat_valid = polyreg.predict(Xp_valid)
print("Degree {:d} Train MSE:      {:.3f}".format(degree, mean_squared_error(yphat_train, y_train)))
print("Degree {:d} Validation MSE: {:.3f}".format(degree, mean_squared_error(yphat_valid, y_valid)))

In [ ]:
xplot = np.linspace(-60,60,100).reshape(-1,1)
xpplot = poly_features.transform(xplot)
yplot = polyreg.predict(xpplot) 
dam_plot([(X_train, y_train, "training"), (X_valid, y_valid, "validation")], [(xplot, yplot, "model")])

**Part C**: Hey!  That looks pretty good! Notice that in addition to an aesthetically pleasing fit, both our training and validation errors went down.  

But we know good things never last ... let's break it.  Go back to the previous part and increment the polynomial degree until the model starts overfitting. 

### Problem 3: Building a Polynomial Regression Pipeline in Scikit-Learn 
***

OK, so our implementation of a polynomial regression model above had a lot of moving parts.  We had to transform both our training and validation data to include polynomial features and then fit a regression model.  Eventually we'll add more bells and whistles to our model as well.  At this point, it's a good idea to talk about how to consolidate this process into a single command using Scikit-Learn's [Pipeline](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) class.  

The general idea is that we'll include all of our transformations and models into a pipeline so we can perform a single call to fit and predict to do the magic.  

**Part A**: Here is how you build a pipeline to generate polynomial features and then fit a regression model at the same time.  Note that you've seen all of these routines before, we're just combining them into one. 


In [ ]:
from sklearn.pipeline import Pipeline

degree = 3
polycombo = [("poly", PolynomialFeatures(degree=degree, include_bias=False)),
                ("linear_regression", LinearRegression())]
polypipe = Pipeline(polycombo)

Now when we call things like `.fit` and `.predict` on our pipeline object, it knows to perform the `.fit` and `.predict` methods for each of the constituents parts in sequence.  Let's try it. Notice that the output after calling `.fit` gives you a nice summary of all of the methods and parameters in your pipeline. 

In [ ]:
polypipe.fit(X_train, y_train)

Finally, we'll use our pipeline to make predictions, check MSEs, and produce a plot.  The workflow will pretty much the same as before, but we've consolidated all of the individual calls to the tranformers and model into the pipeline. 

In [ ]:
yphat_train = polypipe.predict(X_train)
yphat_valid = polypipe.predict(X_valid)
print("Degree {:d} Train MSE:      {:.3f}".format(degree, mean_squared_error(yphat_train, y_train)))
print("Degree {:d} Validation MSE: {:.3f}".format(degree, mean_squared_error(yphat_valid, y_valid)))

xplot = np.linspace(-60,60,100).reshape(-1,1)
yplot = polypipe.predict(xplot) 
dam_plot([(X_train, y_train, "training"), (X_valid, y_valid, "validation")], [(xplot, yplot, "model")])

**Part C**: Let's see how we can systematically zero in on a good polynomial degree. Typically what we'd do is run our model for many values plynomial degree and examine the MSE on the training and validation sets.  The optimal polynomial degree is then the place on the curve with the lowest validation error.  Scikit-Learn implements this functionality using something called a [validation_curve](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.validation_curve.html#sklearn.model_selection.validation_curve). 


In [ ]:
from sklearn.model_selection import validation_curve

# Evaluate the model for degrees 1 through 10 
degrees = np.array(range(1,10+1))

neg_MSE_train_folds, neg_MSE_valid_folds = validation_curve(polypipe, X, y, 
                                                            param_name="poly__degree", param_range=degrees,
                                                            cv=10, scoring="neg_mean_squared_error")

MSE_train = -np.mean(neg_MSE_train_folds, axis=1)
MSE_valid = -np.mean(neg_MSE_valid_folds, axis=1)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(degrees, MSE_train, lw=3, color="steelblue", label="training")
ax.plot(degrees, MSE_valid, lw=3, color="green", label="validation")
ax.grid(alpha=0.25)
ax.set_xlabel("polynomial degree", fontsize=12)
ax.set_ylabel("Error", fontsize=12)
ax.legend(loc="upper right", fontsize=16);


**Part C**: Which value of the polynomial degree appears to be optimal from our validation study? (Also, what's going on with the see-saw pattern for high polynomial degrees?)  Create a new polynomial regression model using the desired polynomial degree, check MSE, make plots, etc. 

In [ ]:
degree = 1 # TODO - Insert best degree here 
best_combo= [("poly", PolynomialFeatures(degree=degree, include_bias=False)),
            ("linear_regression", LinearRegression())]
bestpolypipe = Pipeline(best_combo)

bestpolypipe.fit(X_train, y_train)
yphat_train = bestpolypipe.predict(X_train)
yphat_valid = bestpolypipe.predict(X_valid)
print("Degree {:d} Train MSE:      {:.3f}".format(degree, mean_squared_error(yphat_train, y_train)))
print("Degree {:d} Validation MSE: {:.3f}".format(degree, mean_squared_error(yphat_valid, y_valid)))

xplot = np.linspace(-60,60,100).reshape(-1,1)
yplot = bestpolypipe.predict(xplot) 
dam_plot([(X_train, y_train, "training"), (X_valid, y_valid, "validation")], [(xplot, yplot, "model")])

### Part 5: Evaluating the Bias-Variance using Learning Curves 
***

We've now constructed several different models: a vanilla linear regression model (`linreg`), a general polynomial model with various degrees (`polypipe`), and a polynomial regression model with our empirically determined best degree (`bestpolypipe`). 

**Part A**: What does your intuition tell you with regard to the Bias-Variance Trade-Off of these models.  Does each of them have high or low Bias? Does each of them have high or low variance? 


**Part B**: One way to estimate the bias-variance of a real-life model is by plotting a so-called learning curve.  This is how you plot a learning curve for the `linreg` model in Scikit-Learn. What does the resulting learning curve indicate with respect to bias-variance for the linear model?  

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes, neg_MSE_train_folds, neg_MSE_valid_folds = learning_curve(estimator=linreg, X=X, y=y,
                                                        train_sizes=np.linspace(0.1, 1, 20), cv=10,
                                                        scoring="neg_mean_squared_error") 

MSE_train = -np.mean(neg_MSE_train_folds, axis=1)
MSE_valid = -np.mean(neg_MSE_valid_folds, axis=1)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(train_sizes, MSE_train, lw=3, color="steelblue", label="training")
ax.plot(train_sizes, MSE_valid, lw=3, color="green", label="validation")
ax.grid(alpha=0.25)
ax.set_xlabel("training set size", fontsize=16)
ax.set_ylabel("Error", fontsize=16)
ax.legend(loc="upper right", fontsize=16);

**Part C**: Repeat **Part B** for the degree 10 model `polypipe`. 

In [ ]:
# The validation curve function messed with the polynomial 
# degree of this model, so let's set it to 10 explicitly 
polypipe.set_params(poly__degree=10)

train_sizes = np.array([13+ii for ii in range(43-13)])
train_sizes, neg_MSE_train_folds, neg_MSE_valid_folds = learning_curve(estimator=polypipe, X=X, y=y,
                                                        train_sizes=train_sizes, cv=5,
                                                        scoring="neg_mean_squared_error") 

MSE_train = -np.mean(neg_MSE_train_folds, axis=1)
MSE_valid = -np.mean(neg_MSE_valid_folds, axis=1)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(train_sizes, MSE_train, lw=3, color="steelblue", label="training")
ax.plot(train_sizes, MSE_valid, lw=3, color="green", label="validation")
ax.grid(alpha=0.25)
ax.set_xlabel("training set size", fontsize=16)
ax.set_ylabel("Error", fontsize=16)
ax.legend(loc="upper right", fontsize=16);
ax.set_ylim([0,200]); # Comment this line out if you want to see how bad it really is 

**Part D**: Repeat **Part B** for the optimal degree polynomial model `bestpolypipe`. 

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes = np.array([13+ii for ii in range(43-13)])
train_sizes, neg_MSE_train_folds, neg_MSE_valid_folds = learning_curve(estimator=bestpolypipe, X=X, y=y,
                                                        train_sizes=np.linspace(.28, 1, 20), cv=5, 
                                                        scoring="neg_mean_squared_error") 

MSE_train = -np.mean(neg_MSE_train_folds, axis=1)
MSE_valid = -np.mean(neg_MSE_valid_folds, axis=1)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
ax.plot(train_sizes, MSE_train, lw=3, color="steelblue", label="training")
ax.plot(train_sizes, MSE_valid, lw=3, color="green", label="validation")
ax.grid(alpha=0.25)
ax.set_xlabel("training set size", fontsize=16)
ax.set_ylabel("Error", fontsize=16)
ax.legend(loc="upper right", fontsize=16);

<br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br><br>

### Helper Functions
***

In [ ]:
def dam_plot(scatter=[], models=[]):
    '''
    Function to plot the dam data 
    '''
    
    # colors for scatter plots and model plots 
    scolors = ["steelblue", "#a76c6e", "#6a9373", "orange"]
    mcolors = ["black", "gray"]
    
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8,6))
    
    # Loop over scatter data and make plots 
    for ii, (x, y, label) in enumerate(scatter):
        ax.scatter(x, y, s=100, color=scolors[ii], label=label, zorder=2)
        
    # Loop over model data and make plots 
    for ii, (xplot, yplot, label) in enumerate(models):
        ax.plot(xplot, yplot, color=mcolors[ii], lw=3, label=label, zorder=1)
        
    # Set axis limits
    ax.set_xlim([-60,60])
    ax.set_ylim([-5,60])
        
    # Label all the things 
    ax.set_xlabel("change in water level", fontsize=16)
    ax.set_ylabel("water flowing out of damn", fontsize=16)
    ax.set_title("Dam Data", fontsize=20); ax.grid(alpha=0.25)
    ax.legend(loc="upper left", fontsize=12);